# STEP 0: Install dependencies

In [1]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.8/810.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 13.8 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [2]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [ ]:
os.environ['GOOGLE_API_KEY']=getpass.getpass("enter the Gemini API Key")

# STEP 2: Initialize Gemini 1.5 Flash

In [19]:
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash-latest", temperature =0.2)

# STEP 3: Node to ask user for symptom

In [20]:
def get_symptom(state: dict) ->dict:
  symptom = input("welcome to XYZ hospital, Please enter your symptom")
  state["symptom"] = symptom
  return state

# STEP 4: Node to classify the symptom

In [21]:
def classify_symptom(state: dict) -> dict:
  prompt = (
      "You are a helpful Medical Assistant, Classify the symptoms below into one of the categories \n"
      "-General\n -Emergency \n -mental health \n"
      f"symptom: {state['symptom']}\n"
      "Respond only with one word : Genral, emergency Or Mental Health"
      "#Example :vinput : I have fever, output : General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as : {category}" ) #debug
  state["category"] = category
  return state

# STEP 5: Router logic to route to the correct node


In [22]:
def symptom_router(state:dict) -> dict:
  cat = state["category"].lower()
  if "general" in cat:
    return "general"
  elif "emergency" in cat:
    return "emergency"
  elif "mental" in cat:
    return "mental_health"
  else:
    return"general"
  return END

# STEP 6: Category-specific response nodes


In [23]:
def general_node(state: dict) -> dict:
  state["answer"] = f" '{state['symptom']}' : seems general : directing you to general ward  for consulting a doctor"
  return state

def emergency_node(state:dict) -> dict:
    state["answer"] = f" '{state['symptom']}' : seems emergency : directing you to emergency ward  for consulting a doctor"
    return state

def mental_health_node(state:dict) -> dict:
    state["answer"] = f" '{state['symptom']}' : seems mental health : directing you to mental health ward  for consulting a doctor"
    return state




# STEP 7: Build LangGraph


In [24]:
builder = StateGraph(dict)

builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify_symptom", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.add_edge("get_symptom","classify_symptom")

builder.add_conditional_edges("classify_symptom",symptom_router,{
    "general":"general",
    "emergency":"emergency",
    "mental_health":"mental_health"
})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)


# STEP 8: Compile and invoke the graph


In [25]:
graph = builder.compile()
final_state = graph.invoke({})
print("final Output \n")
print(final_state["answer"])

welcome to XYZ hospital, Please enter your symptomi have high fever
LLM classifies the symptom as : Emergency
final Output 

 'i have high fever' : seems emergency : directing you to emergency ward  for consulting a doctor
